
<a href="https://colab.research.google.com/github/JDBumgardner/Trumpifyve/Trumpifyve_Run_Saved_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### Licensed under the Apache License, Version 2.0 (the "License");

In [0]:
# Copyright 2019 The T5 Authors. All Rights Reserved.
# Copyright 2020 Jacob Bumgardner, Jack Robison, and Google inc. 
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

#### Description 

https://github.com/JDBumgardner/Trumpifyve/

In this notebook we demonstrate how to run inference on our pretrained models to trumpify arbitrary text. First we decide on a model temperature, then we load the saved and exported model and then we apply the style shift over input text. 


## Load the saved model

In [0]:
%tensorflow_version 2.x
!pip install tensorflow-text
from google.colab import auth
auth.authenticate_user()

In [0]:
MODEL_TEMPERATURE = "1.5" #@param["0", "1", "1.5", "2"]
saved_model_path = {
    "0": 'gs://style_shift_trump/models/3B/export/1587870976',
    "1": 'gs://style_shift_trump/models/3B/export/1587870500',
    "1.5":'gs://style_shift_trump/models/3B/export/1587870045',
    "2":'gs://style_shift_trump/models/3B/export/1587869196',
    }[MODEL_TEMPERATURE]


In [0]:
import tensorflow as tf
import tensorflow_text  # Required to run exported model.

def load_predict_fn(model_path):
  if tf.executing_eagerly():
    print("Loading SavedModel in eager mode.")
    imported = tf.saved_model.load(model_path, ["serve"])
    return lambda x: imported.signatures['serving_default'](tf.constant(x))['outputs'].numpy()
  else:
    print("Loading SavedModel in tf 1.x graph mode.")
    tf.compat.v1.reset_default_graph()
    sess = tf.compat.v1.Session()
    meta_graph_def = tf.compat.v1.saved_model.load(sess, ["serve"], model_path)
    signature_def = meta_graph_def.signature_def["serving_default"]
    return lambda x: sess.run(
        fetches=signature_def.outputs["outputs"].name, 
        feed_dict={signature_def.inputs["input"].name: x}
    )

predict_fn = load_predict_fn(saved_model_path)

## Apply

We can now call the style_shift method.

In [0]:
def style_shift(text_sample):
  return predict_fn(["trumpifyve:" + text_sample])[0].decode('utf-8')

list_of_text_samples = [
                        "Encourage your staff or community members to protect their personal health.",
                        "Post the signs and symptoms of COVID-19: fever, cough, shortness of breath.",
                        "Clean surfaces that are frequently touched – things such as shared desks, countertops, kitchen areas, electronics, and doorknobs.",
                        "Limit events and meetings that require close contact.",
                        "Stay up to date on developments in your community.",
                        "Create an emergency plan for possible outbreak.",
                        "Assess if community members are at higher risk and plan accordingly."
                 ]


for text_sample in list_of_text_samples:
    print(style_shift(text_sample))